## Overview

Mistral Large 3 is Mistral’s first mixture-of-experts model since the seminal Mixtral series, and represents a substantial step forward in pretraining at Mistral. After post-training, the model achieves parity with the best instruction-tuned open-weight models on the market on general prompts, while also demonstrating image understanding and best-in-class performance on multilingual conversations (i.e., non-English/Chinese).

Mistral Large 3 debuts at #2 in the OSS non-reasoning models category (#6 amongst OSS models overall) on the <a href="https://lmarena.ai/leaderboard/text">LMArena leaderboard</a> and is released under the Apache 2.0 license.

In this notebook, we will show how some basics on how to work with Mistral Large 3 in Microsoft Foundry.

In [ ]:
import requests
import re
from IPython.display import Markdown, display

In [ ]:
AZURE_AI_FOUNDRY_PROJECT_ENDPOINT = ""
AZURE_AI_FOUNDRY_KEY = ""
AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME = ""

In [ ]:
REQUEST_HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {AZURE_AI_FOUNDRY_KEY}",
}

## Compound questions

In chat and search use cases users can often ask questions that are complex, abstract, or are actually multiple questions within a single query. Mistral Large 3 is able to handle these types of questions and provide comprehensive answers.

In [ ]:
compoundQuestion1 = {
    "model": f"{AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME}",
    "messages": [
        {
            "role": "user",
            "content": "Can you explain the nature of the universe and make an analogy to butterflies?",
        }
    ],
}

In [ ]:
compoundQuestion1Response = requests.post(
    url=AZURE_AI_FOUNDRY_PROJECT_ENDPOINT,
    json=compoundQuestion1,
    headers=REQUEST_HEADERS,
)

In [ ]:
display(Markdown(compoundQuestion1Response.json()["choices"][0]["message"]["content"]))

In [ ]:
compoundQuestion2 = {
    "model": f"{AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME}",
    "messages": [
        {
            "role": "system",
            "content": "You are an expert in Financial Services. Respond in professional and concise language.",
        },
        {
            "role": "user",
            "content": """I recently started a hedge fund in the United States and want to take a client out for dinner, how much can I spend? 
                        Also I am interested in measuring risk in my trades, is there a formula or methodology that can help here?""",
        },
    ],
}

In [ ]:
compoundQuestion2Response = requests.post(
    url=AZURE_AI_FOUNDRY_PROJECT_ENDPOINT,
    json=compoundQuestion2,
    headers=REQUEST_HEADERS,
)

In [ ]:
display(Markdown(compoundQuestion2Response.json()["choices"][0]["message"]["content"]))

## Long Document Understanding

Mistral Large 3 supports a 256k context window. This is useful working with long documents, code bases, or other long-format corporpora.

Source: https://www.gutenberg.org/ebooks/1184

In [ ]:
!wget "https://www.gutenberg.org/cache/epub/1184/pg1184.txt"

In [ ]:
with open("pg1184.txt", "r") as file:
    file_content = file.read()

In [ ]:
# Take the first three volumes of the book
matches = re.finditer(r"VOLUME TWO", file_content, re.IGNORECASE)
if matches:
    second_occurrence = next((m for i, m in enumerate(matches) if i == 1), None)
    if second_occurrence:
        extracted_text = file_content[: second_occurrence.start()].strip()
    else:
        extracted_text = file_content.strip()
else:
    extracted_text = file_content.strip()

In [ ]:
longDocument1 = {
    "model": f"{AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME}",
    "messages": [
        {
            "role": "user",
            "content": f"""an you summarize the following text into a paragraph? Based on this text, is revenge the same thing as justice?  
                        List the main characters and key themes. 
                        Can you explore and expand on how this work can be applied to moderm business and contemporary society? 
                        Content: {extracted_text}""",
        }
    ],
}

In [ ]:
longDocument1Response = requests.post(
    url=AZURE_AI_FOUNDRY_PROJECT_ENDPOINT,
    json=longDocument1,
    headers=REQUEST_HEADERS,
)

In [ ]:
display(Markdown(longDocument1Response.json()["choices"][0]["message"]["content"]))

Source: https://www.gutenberg.org/ebooks/1497

In [ ]:
!wget "https://www.gutenberg.org/cache/epub/1497/pg1497.txt"

In [ ]:
with open("pg1497.txt", "r") as file:
    plato = file.read()

In [ ]:
# Take the first three volumes of the book
matches = re.finditer(r"BOOK I.", plato, re.IGNORECASE)
if matches:
    third_occurrence = next((m for i, m in enumerate(matches) if i == 2), None)
    if third_occurrence:
        extracted_text_plato = plato[: third_occurrence.start()].strip()
    else:
        extracted_text_plato = plato.strip()
else:
    extracted_text_plato = plato.strip()

In [ ]:
longDocument2 = {
    "model": f"{AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME}",
    "messages": [
        {
            "role": "user",
            "content": f"""Can you summarize the following text into a paragraph? Based on this text, what is the allegory of the cave?  
                        What are some key philosophical teachings in this text? 
                        What teachings in this text can be seen with the first Matrix movie? 
                        Content: {extracted_text_plato}""",
        }
    ],
}

In [ ]:
longDocument2Response = requests.post(
    url=AZURE_AI_FOUNDRY_PROJECT_ENDPOINT,
    json=longDocument2,
    headers=REQUEST_HEADERS,
)

In [ ]:
display(Markdown(longDocument2Response.json()["choices"][0]["message"]["content"]))

## Math

Math can be challenging for large language models. Mistral Large 3 is able to to handle a variety of math problems, including those that require complex reasoning and problem solving. This is useful for applications like tutoring, education, and research.

In [ ]:
math1 = {
    "model": f"{AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME}",
    "messages": [
        {
            "role": "user",
            "content": """The World Cup takes place in 2026 but has a limited number of tickets available through a ficticious ticket broker.
                        They are only selling tickets to the Final to customers who have bought at least two tickets to the group stage, one from each round.
                        I have $1000 to spend on tickets altogether, how can I optimize my spending to minimize the amount I have to spend on tickets I have to buy and still qualify to attend the final?
                        Finals ticket cost: $500"
                        Qualifying Round 1:
                        Game A: $350
                        Game B: $400
                        Game C: $300
                        Qualifying Round 2
                        Game D: $175
                        Game E: $250
                        Game F: $300
                        
                        Show your work and thought process""",
        }
    ],
    "temperature": 0.2,
}

In [ ]:
math1Response = requests.post(
    url=AZURE_AI_FOUNDRY_PROJECT_ENDPOINT,
    json=math1,
    headers=REQUEST_HEADERS,
)

In [ ]:
display(Markdown(math1Response.json()["choices"][0]["message"]["content"]))

In [ ]:
math2 = {
    "model": f"{AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME}",
    "messages": [
        {
            "role": "user",
            "content": """How many ways can you arrange the letters in the word 'camembert'? Also what other cheeses have similar letter arrangements?""",
        }
    ],
    "temperature": 0.2,
}

In [ ]:
math2Response = requests.post(
    url=AZURE_AI_FOUNDRY_PROJECT_ENDPOINT,
    json=math2,
    headers=REQUEST_HEADERS,
)

In [ ]:
display(Markdown(math2Response.json()["choices"][0]["message"]["content"]))

## Finance

Financial Services requires both domain knowledge as well as the ability to reason through complex problems. Mistral Large 3 is well-suited to handle ae these types of questions, and with its 256k context window, it can help support a variety of users and AI applications in this industry.

In [ ]:
finance1 = {
    "model": f"{AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME}",
    "messages": [
        {
            "role": "user",
            "content": """What's the difference between EBITDA and EBIT? Can you give me an example?""",
        }
    ],
}

In [ ]:
finance1Response = requests.post(
    url=AZURE_AI_FOUNDRY_PROJECT_ENDPOINT,
    json=finance1,
    headers=REQUEST_HEADERS,
)

In [ ]:
display(Markdown(finance1Response.json()["choices"][0]["message"]["content"]))

In [ ]:
finance2 = {
    "model": f"{AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME}",
    "messages": [
        {
            "role": "system",
            "content": "You are an expert in Financial Services. Respond in professional and concise language.",
        },
        {
            "role": "user",
            "content": """To calculate Earnings + Depreciations I need the WACC. But in order to calculate the WACC I need Earnings and Depreciation. What am I supposed to do?""",
        },
    ],
    "temperature": 0.2,
}

In [ ]:
finance2Response = requests.post(
    url=AZURE_AI_FOUNDRY_PROJECT_ENDPOINT,
    json=finance2,
    headers=REQUEST_HEADERS,
)

In [ ]:
display(Markdown(finance2Response.json()["choices"][0]["message"]["content"]))